# Gender Recognition by Voice | 05 | Tensorflow Neural Network

In [1]:
import obj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline
sns.set_style('whitegrid')

In [3]:
data_scale = obj.load('var/data_scale')

In [4]:
data_scale.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,-4.049248,0.427355,-4.224901,-2.576102,-5.693607,-0.214778,2.293306,1.762946,-0.039083,0.471575,...,-4.049248,-1.812038,-1.097998,0.565959,-1.564205,-0.708404,-1.431422,-1.419137,-1.454772,male
1,-3.841053,0.611669,-3.999293,-2.486885,-5.588987,-0.258485,4.548056,4.433008,-0.065236,0.594431,...,-3.841053,-1.079594,-1.091533,-0.294030,-1.561916,-0.708404,-1.418107,-1.405818,-1.014103,male
2,-3.463066,1.603848,-4.095851,-2.706986,-3.928699,0.909326,6.513656,7.326207,-1.083730,0.398261,...,-3.463066,-1.365368,-1.100397,0.410480,-1.563866,-0.708404,-1.429203,-1.416917,-1.065344,male
3,-0.992157,0.899998,-0.759454,-0.901418,-0.711205,0.632690,-0.449858,-0.240099,1.516383,1.797340,...,-0.992157,-1.666966,-0.988934,-0.294030,-1.195367,-0.708404,-1.273867,-1.261532,0.614286,male
4,-1.530640,1.322561,-1.676948,-1.268395,-0.792029,1.005588,-0.480911,-0.238940,1.708336,2.114740,...,-1.530640,-1.127233,-1.034015,0.260185,-0.221660,-0.708404,0.124154,0.136933,0.289046,male


In [29]:
data_scale_tf = []
for ind, row in data_scale.head().iterrows():
    print(ind)
    print(row)
    #row2
    data_scale_tf.append(row)
#data_scale_tf

0
meanfreq     -4.04925
sd           0.427355
median        -4.2249
Q25           -2.5761
Q75          -5.69361
IQR         -0.214778
skew          2.29331
kurt          1.76295
sp.ent     -0.0390828
sfm          0.471575
mode         -2.14121
centroid     -4.04925
meanfun      -1.81204
minfun         -1.098
maxfun       0.565959
meandom       -1.5642
mindom      -0.708404
maxdom       -1.43142
dfrange      -1.41914
modindx      -1.45477
label            male
Name: 0, dtype: object
1
meanfreq    -3.84105
sd          0.611669
median      -3.99929
Q25         -2.48688
Q75         -5.58899
IQR        -0.258485
skew         4.54806
kurt         4.43301
sp.ent     -0.065236
sfm         0.594431
mode        -2.14121
centroid    -3.84105
meanfun     -1.07959
minfun      -1.09153
maxfun      -0.29403
meandom     -1.56192
mindom     -0.708404
maxdom      -1.41811
dfrange     -1.40582
modindx      -1.0141
label           male
Name: 1, dtype: object
2
meanfreq    -3.46307
sd           1.60385
med

In [42]:
feat_scale_array = data_scale.drop('label',axis=1).as_matrix()
feat_scale_array[0]

array([-4.04924806,  0.4273553 , -4.22490077, -2.57610164, -5.69360723,
       -0.21477826,  2.29330585,  1.76294635, -0.03908279,  0.4715753 ,
       -2.14121031, -4.04924806, -1.81203825, -1.0979981 ,  0.56595854,
       -1.5642046 , -0.70840431, -1.43142165, -1.41913712, -1.45477229])

In [41]:
label_array = data_scale['label'].as_matrix()
label_array

array(['male', 'male', 'male', ..., 'female', 'female', 'female'], dtype=object)

In [52]:
#  male  --> [0,1]
# female --> [1,0]

target_array = []

for label in label_array:
    if label == 'male':
        target = [0,1]
    else:
        target = [1,0]
    target_array.append(target)

In [5]:
import tensorflow as tf

In [148]:
# Learning Parameters
rate   =   0.010  # training rate
epochs =  50      # number of full training cycles
batch  = 100      # number of data points to train per batch

In [149]:
# Network Parameters
n_hidden_1 = 256  # number of nodes in hidden layer 1
n_hidden_2 = 256  # number of nodes in hidden layer 2
n_input    = len(feat_scale_array[0])  # 20
n_classes  = 2
n_samples  = len(feat_scale_array)  # 3168

In [150]:
X = tf.placeholder('float', [None,n_input])
y = tf.placeholder('float', [None,n_classes])

In [151]:
weights = {
    'w1' : tf.Variable(tf.random_normal([n_input,    n_hidden_1])),
    'w2' : tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes ]))
}

In [152]:
biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden_1])),
    'b2' : tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes ]))
}

In [153]:
def multilayer_perceptron(X, weights, biases):
    
    # Hidden Layer 1
    layer1 = tf.matmul(X, weights['w1'])
    layer1 = tf.add(layer1, biases['b1'])
    layer1 = tf.nn.relu(layer1)
    
    # Hidden Layer 2
    layer2 = tf.matmul(layer1, weights['w2'])
    layer2 = tf.add(layer2, biases['b2'])
    layer2 = tf.nn.relu(layer2) 
    
    # Output Layer
    out = tf.matmul(layer2, weights['out'])
    out = tf.add(out, biases['out'])
    
    return out

In [154]:
model = multilayer_perceptron(X, weights, biases)

In [155]:
f_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(model, y))

In [156]:
f_optimizer = tf.train.AdamOptimizer(learning_rate=rate).minimize(f_cost)

In [157]:
init = tf.initialize_all_variables()

In [158]:
s = tf.InteractiveSession()

s.run(init)

for epoch in range(epochs):
   
    cost_avg = 0.
    
    batch_total = int(n_samples/batch)
    
    count = 0
    for batch_i in range(batch_total):
        X_batch_i = feat_scale_array[count : count+batch]
        y_batch_i =     target_array[count : count+batch]
        count += batch
        
        _, cost = s.run([f_optimizer,f_cost], feed_dict={X:X_batch_i, y:y_batch_i})
        
        cost_avg += cost / batch_total
        
    print('Epoch {}: cost={:.4f}'.format(epoch+1, cost_avg))

print('Model completed successfully!')

Epoch 1: cost=613.6398
Epoch 2: cost=517.7230
Epoch 3: cost=25.2130
Epoch 4: cost=18.6093
Epoch 5: cost=9.7713
Epoch 6: cost=8.7467
Epoch 7: cost=6.9812
Epoch 8: cost=6.0803
Epoch 9: cost=5.3085
Epoch 10: cost=4.5552
Epoch 11: cost=3.9749
Epoch 12: cost=2.9694
Epoch 13: cost=2.2598
Epoch 14: cost=2.4583
Epoch 15: cost=3.1150
Epoch 16: cost=3.3205
Epoch 17: cost=1.7000
Epoch 18: cost=1.3578
Epoch 19: cost=1.6927
Epoch 20: cost=1.3635
Epoch 21: cost=0.7567
Epoch 22: cost=0.2653
Epoch 23: cost=0.1359
Epoch 24: cost=0.1087
Epoch 25: cost=0.1143
Epoch 26: cost=0.0837
Epoch 27: cost=0.0722
Epoch 28: cost=0.0597
Epoch 29: cost=0.0476
Epoch 30: cost=0.0400
Epoch 31: cost=0.0374
Epoch 32: cost=0.0274
Epoch 33: cost=0.0160
Epoch 34: cost=0.0079
Epoch 35: cost=0.0117
Epoch 36: cost=0.0000
Epoch 37: cost=0.0000
Epoch 38: cost=0.0000
Epoch 39: cost=0.0000
Epoch 40: cost=0.0000
Epoch 41: cost=0.0000
Epoch 42: cost=0.0000
Epoch 43: cost=0.0000
Epoch 44: cost=0.0000
Epoch 45: cost=0.0000
Epoch 46: cos

In [159]:
correct_target = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
correct_target = tf.cast(correct_target, 'float')

In [160]:
accuracy = tf.reduce_mean(correct_target)

In [161]:
print('Accuracy is {:.2f}%'.format(100*accuracy.eval({X:feat_scale_array, y:target_array})))

Accuracy is 99.94%
